In [1]:
%load_ext pycodestyle_magic
%pycodestyle_on
# %pycodestyle_off -to turn it off

In [2]:
# import all libraries
from minio import Minio
from pyspark import SparkContext
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, to_timestamp, month, year
import requests
import json
import pandas as pd
from binance import Client
from datetime import datetime, date
import os
from dotenv import load_dotenv, dotenv_values
from binance.helpers import date_to_milliseconds, interval_to_milliseconds
from dateutil.relativedelta import relativedelta
from dateutil.parser import parse
load_dotenv()

True

In [3]:
API_KEY = os.getenv("API_KEY")
SECRET_KEY = os.getenv("SECRET_KEY")
MINIO_USER = os.getenv("MINIO_ROOT_USER")
MINIO_PASSWORD = os.getenv("MINIO_ROOT_PASSWORD")
client_binance = Client(API_KEY, SECRET_KEY)

In [4]:
spark = SparkSession.builder \
    .appName("CryptoETL") \
    .getOrCreate()
# Get the SparkContext from the SparkSession
sc = spark.sparkContext
sc._jsc.hadoopConfiguration().set("fs.s3a.access.key", MINIO_USER)#turn into access key in the future 
sc._jsc.hadoopConfiguration().set("fs.s3a.secret.key", MINIO_PASSWORD)
sc._jsc.hadoopConfiguration().set("fs.s3a.endpoint", "http://localhost:9000")
sc._jsc.hadoopConfiguration().set("fs.s3a.connection.ssl.enabled", "true")
sc._jsc.hadoopConfiguration().set("fs.s3a.path.style.access", "true")
sc._jsc.hadoopConfiguration().set("fs.s3a.attempts.maximum", "1")
sc._jsc.hadoopConfiguration().set("fs.s3a.connection.establish.timeout", "5000")
sc._jsc.hadoopConfiguration().set("fs.s3a.connection.timeout", "10000")


'''
sc._jsc.hadoopConfiguration().set("fs.s3a.path.style.access", "true")
sc._jsc.hadoopConfiguration().set("fs.s3a.connection.ssl.enabled", "false")
sc._jsc.hadoopConfiguration().set("fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem")
sc._jsc.hadoopConfiguration().set("fs.s3a.connection.ssl.enabled", "false")'''


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/03/05 22:38:23 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


'\nsc._jsc.hadoopConfiguration().set("fs.s3a.path.style.access", "true")\nsc._jsc.hadoopConfiguration().set("fs.s3a.connection.ssl.enabled", "false")\nsc._jsc.hadoopConfiguration().set("fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem")\nsc._jsc.hadoopConfiguration().set("fs.s3a.connection.ssl.enabled", "false")'

6:67: E261 at least two spaces before inline comment
6:67: E262 inline comment should start with '# '
6:80: E501 line too long (101 > 79 characters)
6:102: W291 trailing whitespace
12:80: E501 line too long (80 > 79 characters)
19:80: E501 line too long (90 > 79 characters)
21:1: W391 blank line at end of file


In [5]:
# tickers = client.get_all_tickers()
#client_binance.get_exchange_info() #has rate limit info
#client_binance.response.headers #has info on current usage

2:1: E265 block comment should start with '# '
3:1: E265 block comment should start with '# '


In [6]:
# start_datetime = datetime.strptime("01-01-2023", '%d-%m-%Y')
# start_date = start_datetime.strftime("%Y-%m-%dT%H:%M:%SZ")

# end_datetime = datetime.strptime("01-01-2024", '%d-%m-%Y')
# end_date_date = end_datetime.strftime("%Y-%m-%dT%H:%M:%SZ")

In [7]:
cryptos = ['BTCUSDT','ETHUSDT','LTCUSDT','BNBUSDT','DOGEUSDT']

1:21: E231 missing whitespace after ','
1:31: E231 missing whitespace after ','
1:41: E231 missing whitespace after ','
1:51: E231 missing whitespace after ','


In [8]:
klines = client_binance.get_historical_klines(
    symbol=symbol, interval=interval, start_str=start_date, end_str=end_date, limit=1000 
)

NameError: name 'symbol' is not defined

2:80: E501 line too long (88 > 79 characters)
2:89: W291 trailing whitespace


In [ ]:
client_binance.response.headers['x-mbx-used-weight']

In [ ]:
client_binance.get_exchange_info()

In [25]:
start_date = "01-01-2019"
end_date = "31-12-2024"
symbol = "BTCUSDT"
interval = "1M"


5:1: W391 blank line at end of file


In [29]:
#create a loop where start date,end date and output data are defined
#in loop says while end date is greater than start date, keep going
#check last value in output data, add last timeframe and use it as new start date
#if client_binance.response.headers['x-mbx-used-weight'] == 5000: sleep 30
#print used weight response head to monitor
#try and except that shows api error codes
#start_datetime = datetime.strptime(start_date, '%d-%m-%Y')
#end_datetime = datetime.strptime(end_date, '%d-%m-%Y')
#implement binance api exception
output = []
try:
    while end_date > start_date:
        data = client_binance.get_historical_klines(
        symbol=symbol, interval=interval, start_str=start_date, end_str=end_date, limit=1000 
        )
        output +=data
        #timeframe = interval_to_milliseconds(interval)
        last_date = data[-1][0]
        #get month and year from last date and compare with end_date month and year
        last_datetime = datetime.utcfromtimestamp(last_date / 1000)
        end_datetime = datetime.strptime(end_date, '%d-%m-%Y')
        if last_datetime.year == end_datetime.year and last_datetime.month == end_datetime.month:
            break #return
        start_datetime =  last_datetime + relativedelta(months=1)
        start_date = str(start_datetime)
        print(client_binance.response.headers['x-mbx-used-weight'])
        #for daily add sleep function
        if client_binance.response.headers['x-mbx-used-weight'] == 5000:
            sleep 30
except Exception as e:
    print("An exception occurred:", type(e).__name__, "–", e)


    

SyntaxError: invalid syntax (982548043.py, line 27)

1:1: E265 block comment should start with '# '
2:1: E265 block comment should start with '# '
3:1: E265 block comment should start with '# '
3:80: E501 line too long (81 > 79 characters)
4:1: E265 block comment should start with '# '
5:1: E265 block comment should start with '# '
6:1: E265 block comment should start with '# '
7:1: E265 block comment should start with '# '
8:1: E265 block comment should start with '# '
13:9: E122 continuation line missing indentation or outdented
13:80: E501 line too long (92 > 79 characters)
13:93: W291 trailing whitespace
15:18: E225 missing whitespace around operator
16:9: E265 block comment should start with '# '
18:9: E265 block comment should start with '# '
18:80: E501 line too long (83 > 79 characters)
21:80: E501 line too long (97 > 79 characters)
22:18: E261 at least two spaces before inline comment
22:19: E262 inline comment should start with '# '
23:25: E222 multiple spaces after operator
26:9: E265 block comment should start with '# '
27:72

Error before execution: invalid syntax (982548043.py, line 27)


In [27]:
output

[[1546300800000,
  '3701.23000000',
  '4069.80000000',
  '3349.92000000',
  '3434.10000000',
  '908244.14054000',
  1548979199999,
  '3320676978.28286698',
  6315903,
  '471134.13749800',
  '1722159891.30143057',
  '0'],
 [1548979200000,
  '3434.10000000',
  '4198.00000000',
  '3373.10000000',
  '3813.69000000',
  '861783.98672700',
  1551398399999,
  '3202849295.15280277',
  5866928,
  '452235.78556200',
  '1681041785.01339350',
  '0'],
 [1551398400000,
  '3814.26000000',
  '4140.00000000',
  '3670.69000000',
  '4103.95000000',
  '787190.48925000',
  1554076799999,
  '3091934416.09679203',
  5549651,
  '416484.15659500',
  '1636152913.80973747',
  '0'],
 [1554076800000,
  '4102.44000000',
  '5600.00000000',
  '4067.00000000',
  '5320.81000000',
  '1126961.31510100',
  1556668799999,
  '5743659745.68371413',
  7878682,
  '575775.32679700',
  '2933079322.35865091',
  '0'],
 [1556668800000,
  '5321.94000000',
  '9074.26000000',
  '5316.20000000',
  '8555.00000000',
  '1498410.02561700',


In [ ]:
# api response structure
"""[
  [
    1499040000000,      // Kline open time
    "0.01634790",       // Open price
    "0.80000000",       // High price
    "0.01575800",       // Low price
    "0.01577100",       // Close price
    "148976.11427815",  // Volume
    1499644799999,      // Kline Close time
    "2434.19055334",    // Quote asset volume
    308,                // Number of trades
    "1756.87402397",    // Taker buy base asset volume
    "28.46694368",      // Taker buy quote asset volume
    "0"                 // Unused field, ignore.
  ]
]"""

In [16]:
columns = [
    "Kline open time",
    "Open Price",
    "High price",
    "Low price",
    "Close Price",
    "Volume",
    "Kline Close time",
    "Quote asset volume",
    "Number of trades",
    "Taker buy base asset volume",
    "Taker buy quote asset volume",
    "Ignore",
]

In [17]:
dataframe = spark.createDataFrame(output, columns)

In [18]:
df = dataframe.withColumn("Kline open time", to_timestamp(col("Kline open time") / 1000))
df = df.withColumnRenamed("Kline open time","datetime")
df = df.select('datetime','Open Price','Close Price','Volume')

1:80: E501 line too long (89 > 79 characters)
2:44: E231 missing whitespace after ','
3:26: E231 missing whitespace after ','
3:39: E231 missing whitespace after ','
3:53: E231 missing whitespace after ','


In [19]:
df.tail(5)

[Row(datetime=datetime.datetime(2024, 8, 1, 1, 0), Open Price='64628.01000000', Close Price='58973.99000000', Volume='1010291.47396000'),
 Row(datetime=datetime.datetime(2024, 9, 1, 1, 0), Open Price='58974.00000000', Close Price='63327.59000000', Volume='734117.07575000'),
 Row(datetime=datetime.datetime(2024, 10, 1, 1, 0), Open Price='63327.60000000', Close Price='70292.01000000', Volume='756010.86343000'),
 Row(datetime=datetime.datetime(2024, 11, 1, 0, 0), Open Price='70292.01000000', Close Price='96407.99000000', Volume='1343559.24219600'),
 Row(datetime=datetime.datetime(2024, 12, 1, 0, 0), Open Price='96407.99000000', Close Price='93576.00000000', Volume='1019450.06577300')]

In [ ]:
from pyspark.sql.functions import year

df_2022 = df.filter(year(df.datetime) == 2024)

In [ ]:
df_2022.show()

In [ ]:
client_minio = Minio(
    "localhost:9000",  # Make sure you're using port 9000 for the S3 API
    access_key = MINIO_USER,
    secret_key = MINIO_PASSWORD,
    secure=False  # Disable SSL if you're not using SSL certificates
)


In [ ]:
buckets = client_minio.list_buckets()

In [ ]:
#df.write.csv('test',header = True)

In [ ]:
#df.coalesce(1).write.csv("csv", header=True)
'''
destination_file = 'csv'
source_file = './csv/part-00000-403543db-3e05-4013-a90b-934dff0f9694-c000.csv' #just an object not technically a file
bucket_name = 'bucket1'
client_minio.fput_object(
        bucket_name, destination_file, source_file,
    )
'''

In [ ]:
bucket_name = 'bucket1'


df = df.withColumn("year", year(df["datetime"])) \
       .withColumn("month", month(df["datetime"]))
df.write.mode("append") \
    .partitionBy("year", "month") \
    .parquet(f"s3a://{bucket_name}/BTC/Daily")